In [ ]:
# Grid Search do parâmetro número de neuronios
Fonte: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout, LSTM, Embedding, Input
from tensorflow import keras
from keras import layers

import pandas as pd
import tensorflow as tf
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.01)

def create_model(neurons):
  model=Sequential()
  model.add(LSTM(neurons, activation = 'relu', input_shape=(lags,1)))
  #model.add(LSTM(neurons_2, activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  ## tamanho do lote e o número de épocas /
  return model

In [3]:
seed = 7
tf.random.set_seed(seed)

In [4]:
data = pd.read_csv('Inteligencia_Artificial/Trabalho de Conclusão de Curso/Base de Dados/2021/BBDC4_30m_2021.csv') 
data.head()

,time,open,high,low,close,Volume,Volume MA
0,2021-01-04T10:00:00-03:00,21.814317,21.814317,21.505287,21.521135,1880012,1384668.55
1,2021-01-04T10:30:00-03:00,21.529059,21.576602,21.441896,21.497363,1532703,1368010.00
2,2021-01-04T11:00:00-03:00,21.505287,21.552830,21.473592,21.481516,877715,1345874.15
3,2021-01-04T11:30:00-03:00,21.497363,21.521135,21.132867,21.188334,5465393,1548509.50
4,2021-01-04T12:00:00-03:00,21.188334,21.227953,21.085324,21.124943,4150374,1694177.35


In [5]:
symbol = "close"
data = pd.DataFrame(data[symbol])
data.head()
data['return'] = np.log(data['close']/data['close'].shift(1))
temp = pd.DataFrame({})
temp['direction'] = np.where(data['return'] > 0, 1, 0)


lags = 4

cols = []
for lag in range(1, lags + 1):
     col = f'lag_{lag}'
     temp[col] = data['return'].shift(lag)
     cols.append(col)
temp.dropna(inplace=True)


temp.index = range(len(temp['direction']))

np.set_printoptions(threshold=False)

def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  Y = df_as_np[:,0]

  middle_matrix = df_as_np[:,1:5]

  X = middle_matrix.reshape((len(Y), middle_matrix.shape[1], 1))

  return X, Y.astype(int)

X, y = windowed_df_to_date_X_y(temp)

#-----------------------------------------------------------
# Define os conjuntos de dados de treinamento, teste e validação
#-----------------------------------------------------------
q_80 = int(len(X) * .8)

X_train, y_train = X[:q_80], y[:q_80]
X_test, y_test = X[q_80:], y[q_80:]




mu, std = X_train.mean(), X_train.std()
X_train = (X_train - mu) / std

X_test = (X_test - mu) / std

print(X_train)
print(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

[[[-0.59602594]
  [-2.76167063]
  [-0.14079378]
  [-0.2149112 ]]

 [[-1.5097588 ]
  [-0.59602594]
  [-2.76167063]
  [-0.14079378]]

 [[-0.37336195]
  [-1.5097588 ]
  [-0.59602594]
  [-2.76167063]]

 ...

 [[ 1.05426085]
  [-0.94370054]
  [ 0.863996  ]
  [-0.84834892]]

 [[ 0.85997355]
  [ 1.05426085]
  [-0.94370054]
  [ 0.863996  ]]

 [[-1.12918969]
  [ 0.85997355]
  [ 1.05426085]
  [-0.94370054]]]
[[[-1.04008709]
  [-1.12918969]
  [ 0.85997355]
  [ 1.05426085]]

 [[-0.66188858]
  [-1.04008709]
  [-1.12918969]
  [ 0.85997355]]

 [[ 1.53532774]
  [-0.66188858]
  [-1.04008709]
  [-1.12918969]]

 ...

 [[-0.51496261]
  [ 0.53060969]
  [ 1.26805957]
  [-0.30797794]]

 [[ 0.00782354]
  [-0.51496261]
  [ 0.53060969]
  [ 1.26805957]]

 [[-0.20167122]
  [ 0.00782354]
  [-0.51496261]
  [ 0.53060969]]]


((3146, 4, 1), (3146,), (787, 4, 1), (787,))

In [6]:
from sklearn.model_selection import TimeSeriesSplit

dev_size=0.2 
n_splits=int((1//dev_size)-1)   # using // for integer division
tscv=TimeSeriesSplit(n_splits=n_splits) 
tscv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)

In [7]:
# create model
# tamanho do lote e o número de épocas / 
model = KerasClassifier(model=create_model, epochs=100, batch_size=40, verbose=0)


In [ ]:
for i in range(30):
  # define the grid search parameters
  neurons = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
  #neurons = [1,2,3,4,5,6,7,8,9,10]
  #neurons_2 = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
  param_grid = dict(model__neurons=neurons)
  #param_grid = dict(model__neurons=neurons_1, model__neurons=neurons_2)
  # https://towardsdatascience.com/cross-validation-and-grid-search-efa64b127c1b (CV parameter)
  # https://scikit-learn.org/stable/modules/cross_validation.html 
  #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
  #https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=tscv)
  grid_result = grid.fit(X_train, y_train)


  #Precisão
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

## 1 teste: 